In [1]:
%%javascript
Jupyter.utils.load_extensions('tdb_ext/main')

<IPython.core.display.Javascript object>

In [2]:
# coding: utf-8
import tensorflow as tf
import prettytensor as pt
import numpy as np
import cmtf.data.data_mnist as data_mnist
import matplotlib.pyplot as plt
import tdb
from tdb.examples import viz


# 模型
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int64, [None])

x_reshape = tf.reshape(x, [-1, 28, 28, 1])
seq = pt.wrap(x_reshape).sequential()
seq.conv2d([7, 7], 16)
seq.max_pool(2, 2)
seq.conv2d([7, 7], 32)
seq.max_pool(2, 2)
seq.flatten()
seq.fully_connected(64, activation_fn=tf.nn.relu)
seq.fully_connected(10, activation_fn=None)
output = seq.as_layer()

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(output, y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output,1), y), "float"))
train_op = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

# 所有变量
for v in tf.all_variables():
	print v.name

# GPU使用率
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8    #固定比例
config.gpu_options.allow_growth = True

# session
sess = tf.InteractiveSession(config=config)
tf.initialize_all_variables().run()

g=tf.get_default_graph()
conv1_weights = [v for v in tf.all_variables() if v.name == "conv2d/weights:0"][0]
p1=tdb.plot_op(viz.viz_conv_weights,inputs=[g.as_graph_element(conv1_weights)])

# 数据
mnist = data_mnist.read_data_sets(one_hot=False)

# train
for i in range(2000):
	batch_xs, batch_ys = mnist.train.next_batch(32)
	train_op.run(feed_dict={x: batch_xs, y: batch_ys})
	if (i+1)%200 == 0:
		feed_dict={x: batch_xs, y: batch_ys}
		status, result=tdb.debug([loss, p1], feed_dict=feed_dict, breakpoints=None, break_immediately=False, session=sess)
		print 'index: %d, loss: %f' % (i+1, result[0])
# test
accuracy_value = accuracy.eval(feed_dict={x:mnist.test.images[:100], y:mnist.test.labels[:100]})
print 'Accuracy: %g' % accuracy_value












conv2d/weights:0
conv2d/bias:0
conv2d_1/weights:0
conv2d_1/bias:0
fully_connected/weights:0
fully_connected/bias:0
fully_connected_1/weights:0
fully_connected_1/bias:0
Extracting /Users/lichen/data/tf/mnist/train-images-idx3-ubyte.gz
Extracting /Users/lichen/data/tf/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/lichen/data/tf/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/lichen/data/tf/mnist/t10k-labels-idx1-ubyte.gz
index: 200, loss: 8.306932
index: 400, loss: 6.714475
index: 600, loss: 5.983556
index: 800, loss: 5.558240
index: 1000, loss: 5.074481
index: 1200, loss: 3.636143
index: 1400, loss: 0.870659
index: 1600, loss: 1.979830
index: 1800, loss: 1.218139
index: 2000, loss: 2.785712
Accuracy: 0.83
